# g) Ngonye Falls Power and Energy Periods
Produces 'simulations' of station operation over two periods, debt period and equity period. Each simulation has a different start date on the historic series.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
|    [Model Name]_pe_yearly.csv| Notebook: d_ewr_setup | Summary energy produciton by year of the historic series |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
| [Model Name]_pe_sims.csv | Detailed (yearly) output. |
| [Model Name]_pe_sims_summary.csv | Summary by start year  |



## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [2]:
input_data='./input_data/'
output_data='./output_data/'

model_name='Base Case'


In [3]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix

debt_period=18
equity_period=25
discount_rate=1.0 + 0.15

In [4]:
yearly=pd.read_csv(output_data + out_prefix + "_pe_yearly.csv")

In [5]:
yearly

,WaterYear,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,...,PropDays4uPotentialGtRatedPower,FstDay1uPotentialLessRatedPower,LstDay1uPotentialLessRatedPower,FstDay2uPotentialLessRatedPower,LstDay2uPotentialLessRatedPower,FstDay3uPotentialLessRatedPower,LstDay3uPotentialLessRatedPower,FstDay4uPotentialLessRatedPower,LstDay4uPotentialLessRatedPower,Exceedance
0,1924,3452.248382,111.407480,0.870678,0.053009,25.405610,13.543810,59.190627,974.519288,964.401488,...,246,58.0,156.0,57.0,158.0,56.0,161.0,29.0,165.0,0.60
1,1925,4501.386001,158.541897,0.484050,0.040654,25.413935,11.465562,60.263918,976.597536,964.401488,...,261,67.0,157.0,66.0,158.0,65.0,162.0,63.0,166.0,0.85
2,1926,3313.040641,192.895237,0.497958,0.065499,25.434577,13.838798,59.094116,974.224300,964.401488,...,242,54.0,160.0,52.0,162.0,49.0,164.0,45.0,167.0,0.35
3,1927,2261.108088,179.826510,0.472707,0.065455,25.389344,16.332837,59.616057,971.730261,964.401815,...,268,68.0,153.0,67.0,154.0,66.0,157.0,64.0,161.0,0.60
4,1928,1589.152359,195.207704,0.498797,0.093131,25.385336,18.290452,60.263918,969.772646,964.401488,...,325,107.0,124.0,103.0,128.0,100.0,131.0,96.0,135.0,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2012,3614.802471,223.273423,0.508220,0.055826,25.246787,13.199350,60.037088,974.863748,964.539641,...,227,24.0,154.0,24.0,155.0,23.0,156.0,21.0,158.0,0.45
89,2013,3574.946119,195.061590,0.497279,0.056428,25.434562,13.283807,59.188683,974.779291,964.400079,...,215,47.0,181.0,46.0,182.0,44.0,185.0,39.0,188.0,0.10
90,2014,1460.253290,208.780681,0.546606,0.099878,25.352568,18.751821,60.094814,969.356203,964.452684,...,283,132.0,146.0,126.0,149.0,89.0,156.0,81.0,162.0,0.25
91,2015,2975.524395,176.218173,0.496598,0.067347,25.389339,14.608384,59.876605,973.454714,964.400079,...,254,61.0,162.0,61.0,162.0,60.0,165.0,58.0,169.0,0.80


In [6]:
sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims['SimName']=sims['StartYear']
sims['EquivalentPeriod']=2027-sims['StartYear']-2
sims=sims.set_index('StartYear')
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod
StartYear,,,,
1924,1941,1948,1924,101
1925,1942,1949,1925,100
1926,1943,1950,1926,99
1927,1944,1951,1927,98
1928,1945,1952,1928,97
...,...,...,...,...
1988,2005,2012,1988,37
1989,2006,2013,1989,36
1990,2007,2014,1990,35


In [7]:
sims.loc[1967]

DebtEnd             1984
EquityEnd           1991
SimName             1967
EquivalentPeriod      58
Name: 1967, dtype: int64

In [8]:


o=pd.DataFrame()
for row in sims.itertuples(): 
    f=pd.DataFrame(yearly.loc[(yearly['WaterYear']>=row.Index) & (yearly['WaterYear']<=row.EquityEnd)].loc[:,['Energy']])
    f['Energy']=f['Energy']/1000
    f=f.reset_index()
    f['YearNo']=f.index+1
    f['StartYear']=row.Index
    f['WaterYear']=f['StartYear']+f.index
    f=f.set_index('StartYear').drop('index',axis=1)
    #display(f)
    
    o=o.append(f)

sims=sims.join(o)
#sims=sims.reset_index()
#sims=sims.set_index(['StartYear','YearNo'])
sims
#o

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear
StartYear,,,,,,,
1924,1941,1948,1924,101,830.191215,1,1924
1924,1941,1948,1924,101,770.055941,2,1925
1924,1941,1948,1924,101,861.916711,3,1926
1924,1941,1948,1924,101,828.253643,4,1927
1924,1941,1948,1924,101,746.900555,5,1928
...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,845.323095,21,2012
1992,2009,2016,1992,33,928.592840,22,2013
1992,2009,2016,1992,33,885.147984,23,2014


In [9]:
flow_yearly=pd.read_csv(output_data + "ngonye_flow_yearly.csv").set_index('WaterYear')
sims['Flow_mean']=sims.join(flow_yearly,on='WaterYear',lsuffix='l_')['Flow_mean']
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean
StartYear,,,,,,,,
1924,1941,1948,1924,101,830.191215,1,1924,992.089057
1924,1941,1948,1924,101,770.055941,2,1925,1111.873684
1924,1941,1948,1924,101,861.916711,3,1926,990.425137
1924,1941,1948,1924,101,828.253643,4,1927,848.055028
1924,1941,1948,1924,101,746.900555,5,1928,564.473244
...,...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,845.323095,21,2012,1336.562700
1992,2009,2016,1992,33,928.592840,22,2013,1340.578429
1992,2009,2016,1992,33,885.147984,23,2014,697.100110


In [10]:
sims['CumEnergy']=sims['Energy'].groupby('StartYear').cumsum()

In [11]:
sims['DiscountedEnergy']=sims['Energy']*(1/(discount_rate**(sims['YearNo']-1)))

In [12]:
sims['CumDiscountedEnergy']=sims['DiscountedEnergy'].groupby('StartYear').cumsum()

In [13]:
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean,CumEnergy,DiscountedEnergy,CumDiscountedEnergy
StartYear,,,,,,,,,,,
1924,1941,1948,1924,101,830.191215,1,1924,992.089057,830.191215,830.191215,830.191215
1924,1941,1948,1924,101,770.055941,2,1925,1111.873684,1600.247156,669.613862,1499.805077
1924,1941,1948,1924,101,861.916711,3,1926,990.425137,2462.163867,651.732863,2151.537939
1924,1941,1948,1924,101,828.253643,4,1927,848.055028,3290.417510,544.590215,2696.128155
1924,1941,1948,1924,101,746.900555,5,1928,564.473244,4037.318066,427.042817,3123.170971
...,...,...,...,...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,845.323095,21,2012,1336.562700,16677.568912,51.649477,5345.488831
1992,2009,2016,1992,33,928.592840,22,2013,1340.578429,17606.161752,49.336767,5394.825597
1992,2009,2016,1992,33,885.147984,23,2014,697.100110,18491.309736,40.894358,5435.719956


In [14]:
sims_summary=(sims.loc[sims['YearNo']==debt_period]).drop(['Energy','YearNo','DiscountedEnergy','DebtEnd','EquityEnd'],axis=1).rename(columns={'CumEnergy':'DebtEnergy','CumDiscountedEnergy':'DebtDiscountedEnergy'})

sims_summary=sims_summary.join((sims.loc[sims['YearNo']==equity_period])
            .drop(['Energy','YearNo','DiscountedEnergy','SimName','EquivalentPeriod','WaterYear','Flow_mean'],axis=1)
            .rename(columns={'CumEnergy':'EquityEnergy','CumDiscountedEnergy':'EquityDiscountedEnergy'}))

sims_summary.head(3)

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy
StartYear,,,,,,,,,,
1924,1924,101,1941,649.518919,14859.849551,5786.797359,1941,1948,20404.456878,6097.587768
1925,1925,100,1942,669.368439,14839.277220,5775.331619,1942,1949,20331.298962,6083.952451
1926,1926,99,1943,927.855019,14916.626142,5834.812877,1943,1950,20408.430644,6140.576879


In [15]:
sims_summary['DebtAvEnergy']=sims.loc[sims['YearNo']<=debt_period]['Energy'].groupby('StartYear').mean()

In [16]:
sims_summary['EquityAvEnergy']=sims.loc[sims['YearNo']<=equity_period]['Energy'].groupby('StartYear').mean()

In [17]:
sims_summary['DebtAvFlow']=sims.loc[sims['YearNo']<=debt_period]['Flow_mean'].groupby('StartYear').mean()
sims_summary['EquityAvFlow']=sims.loc[sims['YearNo']<=equity_period]['Flow_mean'].groupby('StartYear').mean()

In [18]:
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow
StartYear,,,,,,,,,,,,,,
1924,1924,101,1941,649.518919,14859.849551,5786.797359,1941,1948,20404.456878,6097.587768,825.547197,816.178275,924.969173,932.862973
1925,1925,100,1942,669.368439,14839.277220,5775.331619,1942,1949,20331.298962,6083.952451,824.404290,813.251958,907.040250,948.130812
1926,1926,99,1943,927.855019,14916.626142,5834.812877,1943,1950,20408.430644,6140.576879,828.701452,816.337226,896.816991,952.625050
1927,1927,98,1944,996.690083,14851.226721,5792.847667,1944,1951,20351.140603,6098.568249,825.068151,814.045624,897.165043,976.692444
1928,1928,97,1945,680.775211,14895.206385,5790.336181,1945,1952,20332.931227,6089.160112,827.511466,813.317249,887.871720,1002.207598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1988,37,2005,960.131883,13705.229271,5260.286996,2005,2012,19735.894580,5592.492299,761.401626,789.435783,826.412437,1002.032194
1989,1989,36,2006,1457.112425,13698.560833,5156.608258,2006,2013,19820.552160,5493.280320,761.031157,792.822086,833.605851,1002.550092
1990,1990,35,2007,1300.199178,13848.266154,5154.851672,2007,2014,19959.149534,5489.661177,769.348120,798.365981,876.126697,1009.041138


In [19]:
fdc=pd.read_csv(output_data + "ngonye_flow_annual_exceedance.csv")
sims_summary['DebtEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('DebtAvFlow'),fdc.sort_values('Mean'),left_on='DebtAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary['EquityEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('EquityAvFlow'),fdc.sort_values('Mean'),left_on='EquityAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow,DebtEqExceed,EquityEqExceed
StartYear,,,,,,,,,,,,,,,,
1924,1924,101,1941,649.518919,14859.849551,5786.797359,1941,1948,20404.456878,6097.587768,825.547197,816.178275,924.969173,932.862973,0.653,0.649
1925,1925,100,1942,669.368439,14839.277220,5775.331619,1942,1949,20331.298962,6083.952451,824.404290,813.251958,907.040250,948.130812,0.657,0.636
1926,1926,99,1943,927.855019,14916.626142,5834.812877,1943,1950,20408.430644,6140.576879,828.701452,816.337226,896.816991,952.625050,0.660,0.630
1927,1927,98,1944,996.690083,14851.226721,5792.847667,1944,1951,20351.140603,6098.568249,825.068151,814.045624,897.165043,976.692444,0.660,0.587
1928,1928,97,1945,680.775211,14895.206385,5790.336181,1945,1952,20332.931227,6089.160112,827.511466,813.317249,887.871720,1002.207598,0.662,0.551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1988,37,2005,960.131883,13705.229271,5260.286996,2005,2012,19735.894580,5592.492299,761.401626,789.435783,826.412437,1002.032194,0.708,0.551
1989,1989,36,2006,1457.112425,13698.560833,5156.608258,2006,2013,19820.552160,5493.280320,761.031157,792.822086,833.605851,1002.550092,0.704,0.551
1990,1990,35,2007,1300.199178,13848.266154,5154.851672,2007,2014,19959.149534,5489.661177,769.348120,798.365981,876.126697,1009.041138,0.669,0.547


In [20]:
sims_summary.to_csv(output_data + out_prefix + "_pe_sims_summary.csv")

In [21]:
sims.to_csv(output_data + out_prefix + "_pe_sims.csv")